In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# packs = !ls /kaggle/input/imageai-packages/packages
# for pack in packs:
#     !pip install /kaggle/input/imageai-packages/packages/{pack}

In [ ]:
# !cp ../input/resnet50/resnet50_coco_best_v2.1.0.h5 resnet50_coco_best_v2.1.0.h5

In [ ]:
# !mkdir train && mkdir train/class_1

In [ ]:
# !cp -r ../input/petfinder-pawpularity-score/train/*.jpg train/class_1/

In [ ]:
# !mv train/class_1/train/*.jpg train/class_1/

In [ ]:
# from imageai.Detection import ObjectDetection
# detector = ObjectDetection()
# detector.setModelTypeAsRetinaNet()
# detector.setModelPath("/kaggle/working/resnet50_coco_best_v2.1.0.h5")
# detector.loadModel()

In [ ]:
from PIL import Image
import numpy as np
import os
from tqdm import tqdm
import pandas as pd
# from tensorflow.keras.preprocessing.image import load_img
import warnings
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

import tensorflow as tf
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, Concatenate, BatchNormalization
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomContrast
from tensorflow.keras import Input
from tensorflow.keras.models import load_model
from tensorflow.keras.applications import ResNet50



data_path = '../input/petfinder-pawpularity-score'
train_data_path = os.path.join(data_path, 'train')

test_data_path = os.path.join(data_path, 'test')
test_data_files = os.listdir(test_data_path)

df_true = pd.read_csv(os.path.join(data_path, 'train.csv'))
df_test = pd.read_csv(os.path.join(data_path, 'test.csv'))
sample_submission = pd.read_csv(os.path.join(data_path, 'sample_submission.csv'))


execution_path = os.getcwd()

# pd.cut(df_true['Pawpularity'], bins=10).value_counts().sort_index().plot(kind="bar")

df_true['Id'] = df_true['Id'].apply(lambda x: f'{x}.jpg')
df_true['Pawpularity'] = df_true['Pawpularity'] / 100.0
df_test['Id'] = df_test['Id'].apply(lambda x: f'{x}.jpg')

# x_train = np.asarray([])
# x_train, y_train = [], []
# for data in tqdm(data_files[:np.floor(len(data_files) / 2).astype(int)]):
#     image = Image.open(os.path.join(data_path, data))
# #     image = load_img(os.path.join(data_path, data))
#     image = image.convert('L')
# #     image = image.resize((64, 64), Image.ANTIALIAS)
#     x_train.append(np.asarray(image) / 255.0)
#     y_train.append(df_true[df_true['File'] == data]['Label'].values.astype(np.uint8))

x_train_indices, x_val_indices, y_train_indices, y_val_indices = train_test_split(df_true['Id'], df_true['Pawpularity'], test_size=0.05, random_state=441)
partition = {}
partition['train'] = x_train_indices.values
partition['validation'] = x_val_indices.values
labels = {v['Id']:v['Pawpularity'] for (k, v) in df_true.transpose().to_dict().items()}

In [ ]:
# test_image_id = df_true['Id'].sample(1).values[0]
# test_image = os.path.join(train_data_files, test_image_id)
# extracted_images = detector.detectObjectsFromImage(input_image=test_image, output_image_path=os.path.join(execution_path , f"imagenew.jpg"), extract_detected_objects=True)

In [ ]:
# p = os.listdir(os.path.join(execution_path, '645a92a29b7f0295636bd8604caa545f.jpg_imagenew.jpg-objects'))[0]
# img = Image.open(p)
# img

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def preprocess_image(image):
    image = tf.image.resize_with_pad(image, 224, 224, method="bilinear", antialias=False)
    return image

batch_size=32
datagen_args = dict(rotation_range=20,
                    width_shift_range=0.2,
                    height_shift_range=0.2,
                    horizontal_flip=True,
                    vertical_flip=True,
                    rescale=1./255,
                    validation_split=0.01)

datagen = ImageDataGenerator(**datagen_args)

traingenerator = datagen.flow_from_dataframe(df_true,
                                            train_data_path,
                                            x_col="Id",
                                            y_col="Pawpularity",
                                            target_size=(224,224),
                                            batch_size=batch_size,
                                            interpolation="bilinear",
                                            class_mode="raw",
                                            subset="training",
                                            shuffle=True)

valgenerator = datagen.flow_from_dataframe(df_true,
                                            train_data_path,
                                            x_col="Id",
                                            y_col="Pawpularity",
                                            target_size=(224,224),
                                            batch_size=batch_size,
                                            interpolation="bilinear",
                                            class_mode="raw",
                                            subset="validation",
                                            shuffle=True)

In [ ]:
# test_model = tf.keras.models.Sequential(
#     [
# #         tf.keras.layers.Input(shape=(224, 224, 3)),
#         tf.keras.layers.RandomFlip(),
#         tf.keras.layers.RandomRotation(180),
#         tf.keras.layers.RandomContrast(.2)
#     ]
# )

# plt.imshow(test_model(test_x[0][0][0]))
# test_model.compile(loss="mean_squared_error", optimizer=opt, 
#               metrics=[tf.keras.metrics.RootMeanSquaredError()])
# l = test_model.predict(test_x[0][0])

In [ ]:
# cnn_model = tf.keras.applications.ResNet50V2(
#     include_top=False,
#     weights="imagenet",
#     input_shape=(224, 224, 3),
#     pooling="max",
# )

# !pip install --quiet vit-keras

# from vit_keras import vit

# vit_model = vit.vit_b32(
#         image_size = 224,
#         activation = 'softmax',
#         pretrained = True,
#         include_top = False,
#         pretrained_top = False,
#         classes = 5)

vit_model = load_model('../input/vit-32/vit_b32')
cnn_model = load_model('../input/d/szlky1234/resnet50/resnet50')

In [ ]:
vit_model.trainable = False
cnn_model.trainable = False

In [ ]:

inputs = Input((224, 224, 3))
# augmentations = RandomFlip() (inputs)
# augmentations = RandomRotation(180) (augmentations)
# # augmentations = RandomContrast(.2) (augmentations)


left_side = vit_model(inputs, training=False)
right_side = cnn_model(inputs, training=False)

# left_side = Dense(768, activation="relu") (left_side)
# right_side = Dense(1280, activation="relu") (right_side)

concat = tf.keras.layers.concatenate([left_side, right_side])
concat = BatchNormalization() (concat)
concat = Dense(4096, activation="relu") (concat)
concat = BatchNormalization() (concat)
concat = Dropout(0.2) (concat)
concat = Dense(256, activation="relu") (concat)
concat = Dropout(0.2) (concat)
concat = Dense(1, activation="sigmoid") (concat)

model = Model(inputs=[inputs], outputs=concat)
model.summary()


In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
# opt = tf.keras.optimizers.SGD(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=opt, 
              metrics=[tf.keras.metrics.RootMeanSquaredError()])

# model.compile(loss="mean_squared_error", optimizer=opt, 
#               metrics=[tf.keras.metrics.BinaryCrossentropy()])

In [ ]:
# reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_root_mean_squared_error',
#                                                  factor = 0.2,
#                                                  patience = 2,
#                                                  verbose = 1,
#                                                  min_delta = 1e-4,
#                                                  min_lr = 1e-6,
#                                                  mode = 'min')

earlystopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_root_mean_squared_error',
                                                 min_delta = 0.005,
                                                 patience = 5,
                                                 mode = 'min',
                                                 restore_best_weights = True,
                                                 verbose = 1)
EPOCHS=20
callbacks = [earlystopping]

In [ ]:
# Train model on dataset
model.fit(x=traingenerator,
          validation_data=valgenerator,
          epochs=EPOCHS,
          max_queue_size=20,
          workers=16,
          callbacks=callbacks)
#           class_weight=class_weights) 

In [ ]:
vit_model.trainable = True
cnn_model.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),  
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.RootMeanSquaredError()])

earlystopping_tune = tf.keras.callbacks.EarlyStopping(monitor = 'val_root_mean_squared_error',
                                                     min_delta = 0.001,
                                                     patience = 5,
                                                     mode = 'min',
                                                     restore_best_weights = True,
                                                     verbose = 1)

callbacks = [earlystopping_tune]

In [ ]:
model.fit(x=traingenerator,
          validation_data=valgenerator,
          epochs=EPOCHS,
          max_queue_size=20,
          workers=16, 
          callbacks=callbacks)

In [ ]:
testgenerator = datagen.flow_from_dataframe(df_test,
                                            test_data_path,
                                            x_col="Id",
                                            target_size=(224,224),
                                            batch_size=1,
                                            interpolation="bilinear",
                                            class_mode=None,
                                            shuffle=False)

In [ ]:
files = testgenerator.filenames

In [ ]:
predictions = model.predict(testgenerator)
predictions *= 100
predictions

In [ ]:
sample_submission = pd.DataFrame({"Id": files, "Pawpularity": np.squeeze(predictions)})
sample_submission['Id'] = sample_submission['Id'].apply(lambda x: x[:-4])
sample_submission.head(50)


In [ ]:
sample_submission.to_csv("submission.csv", index=False)